In [1]:
import sys
assert sys.version_info >= (3, 5)

import sklearn
assert sklearn.__version__ >= "0.20"

import numpy as np
import os

import matplotlib as mpl
import matplotlib.pyplot as plt
mpl.rc('axes', labelsize=14)
mpl.rc('xtick', labelsize=12)
mpl.rc('ytick', labelsize=12)

import warnings
warnings.filterwarnings(action="ignore", message="^internal gelsd")

In [2]:
import pandas as pd

def load_housing_data(housing_path='data'):
    train_csv_path = os.path.join(housing_path, "train.csv")
    test_csv_path = os.path.join(housing_path, "test.csv")
    return pd.read_csv(train_csv_path), pd.read_csv(test_csv_path)

train, test = load_housing_data()

In [3]:
train['TotalLivingSF'] = train['BsmtFinSF1'] + train['BsmtFinSF2'] + train['1stFlrSF'] + train['2ndFlrSF']
test['TotalLivingSF'] = test['BsmtFinSF1'] + test['BsmtFinSF2'] + test['1stFlrSF'] + test['2ndFlrSF']

# train['TotalLivingSF'] = np.log(train['TotalLivingSF'])
# test['TotalLivingSF'] = np.log(test['TotalLivingSF'])

train['MSSubClass'] = train['MSSubClass'].apply(str)
train['YrSold'] = train['YrSold'].astype(str)
train['MoSold'] = train['MoSold'].astype(str)

test['MSSubClass'] = test['MSSubClass'].apply(str)
test['YrSold'] = test['YrSold'].astype(str)
test['MoSold'] = test['MoSold'].astype(str)

train['Electrical'] = train['Electrical'].fillna("SBrkr")

train['FireplaceQu'] = train['FireplaceQu'].fillna("None") 
test['FireplaceQu'] = test['FireplaceQu'].fillna("None") 

train['TotalPorchSF'] = train['OpenPorchSF'] + train['3SsnPorch'] + train['EnclosedPorch'] + train['ScreenPorch'] + train['WoodDeckSF']
test['TotalPorchSF'] = test['OpenPorchSF'] + test['3SsnPorch'] + test['EnclosedPorch'] + test['ScreenPorch'] + test['WoodDeckSF']

test['SaleType'] = test['SaleType'].fillna(test['SaleType'].mode()[0])

test['KitchenQual'] = test['KitchenQual'].fillna("TA") 

bsmt_cat_att = ['BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2']

for a in bsmt_cat_att:
    train[a].fillna('NB', inplace=True)
    test[a].fillna('NB', inplace=True)

test['Exterior1st'] = test['Exterior1st'].fillna(test['Exterior1st'].mode()[0])

test['MSZoning'] = test.groupby('MSSubClass')['MSZoning'].transform(lambda x: x.fillna(x.mode()[0]))

train['Alley'] = train['Alley'].fillna("NA")
test['Alley'] = test['Alley'].fillna("NA")

train['Has2ndFloor'] = train['2ndFlrSF'].apply(lambda x: 1 if x > 0 else 0)
test['Has2ndFloor'] = test['2ndFlrSF'].apply(lambda x: 1 if x > 0 else 0)

test['Exterior2nd'] = test['Exterior2nd'].fillna(test['Exterior2nd'].mode()[0])

train['MasVnrType'] = train['MasVnrType'].fillna(train['MasVnrType'].mode()[0])
test['MasVnrType'] = test['MasVnrType'].fillna(test['MasVnrType'].mode()[0])

garage_cat_att = ['GarageType', 'GarageFinish', 'GarageQual', 'GarageCond']

for a in garage_cat_att:
    train[a].fillna('NG', inplace=True)
    test[a].fillna('NG', inplace=True)
    

train['Fence'] = train['Fence'].fillna("NF")
test['Fence'] = test['Fence'].fillna("NF")

test['Functional'] = test['Functional'].fillna("Typ")

train['MiscFeature'] = train['MiscFeature'].fillna("None")
test['MiscFeature'] = test['MiscFeature'].fillna("None")

In [4]:
num_att = list(train.select_dtypes([np.number]).columns)
for a in num_att:
    median = train[a].median()
    train[a].fillna(0, inplace=True)
    
#     median = test[a].median()
    if a != 'SalePrice':
        test[a].fillna(0, inplace=True)

In [5]:
from scipy.stats import skew  # for some statistics
from scipy.special import boxcox1p, inv_boxcox1p
from scipy.stats import boxcox_normmax

numeric_dtypes = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
numerics2 = []
for i in train.columns:
    if train[i].dtype in numeric_dtypes:
        numerics2.append(i)
skew_features = train[numerics2].apply(lambda x: skew(x)).sort_values(ascending=False)

high_skew = skew_features[skew_features > 0.5]
skew_index = high_skew.index
# print(skew_index)
print(high_skew)

MiscVal          24.451640
PoolArea         14.813135
LotArea          12.195142
3SsnPorch        10.293752
LowQualFinSF      9.002080
KitchenAbvGr      4.483784
BsmtFinSF2        4.250888
ScreenPorch       4.117977
BsmtHalfBath      4.099186
EnclosedPorch     3.086696
MasVnrArea        2.674865
OpenPorchSF       2.361912
TotalLivingSF     2.161744
SalePrice         1.880941
BsmtFinSF1        1.683771
WoodDeckSF        1.539792
TotalBsmtSF       1.522688
1stFlrSF          1.375342
GrLivArea         1.365156
TotalPorchSF      1.101180
BsmtUnfSF         0.919323
2ndFlrSF          0.812194
OverallCond       0.692355
TotRmsAbvGrd      0.675646
HalfBath          0.675203
Fireplaces        0.648898
BsmtFullBath      0.595454
dtype: float64


In [6]:
corr_matrix = train.corr()
corr_matrix["SalePrice"].sort_values(ascending=False)

SalePrice        1.000000
OverallQual      0.790982
GrLivArea        0.708624
TotalLivingSF    0.708047
GarageCars       0.640409
GarageArea       0.623431
TotalBsmtSF      0.613581
1stFlrSF         0.605852
FullBath         0.560664
TotRmsAbvGrd     0.533723
YearBuilt        0.522897
YearRemodAdd     0.507101
MasVnrArea       0.472614
Fireplaces       0.466929
TotalPorchSF     0.390993
BsmtFinSF1       0.386420
WoodDeckSF       0.324413
2ndFlrSF         0.319334
OpenPorchSF      0.315856
HalfBath         0.284108
LotArea          0.263843
GarageYrBlt      0.261366
BsmtFullBath     0.227122
BsmtUnfSF        0.214479
LotFrontage      0.209624
BedroomAbvGr     0.168213
Has2ndFloor      0.137656
ScreenPorch      0.111447
PoolArea         0.092404
3SsnPorch        0.044584
BsmtFinSF2      -0.011378
BsmtHalfBath    -0.016844
MiscVal         -0.021190
Id              -0.021917
LowQualFinSF    -0.025606
OverallCond     -0.077856
EnclosedPorch   -0.128578
KitchenAbvGr    -0.135907
Name: SalePr

In [7]:
from scipy.stats import skew  # for some statistics
from scipy.special import boxcox1p
from scipy.stats import boxcox_normmax
from numpy import log1p, expm1

numeric_dtypes = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
numerics2 = []
for i in train.columns:
    if train[i].dtype in numeric_dtypes:
        numerics2.append(i)
skew_features = train[numerics2].apply(lambda x: skew(x)).sort_values(ascending=False)

high_skew = skew_features[skew_features > 0.5]
skew_index = high_skew.index
print(skew_index)
print(high_skew)

for i in skew_index:
    if i != 'SalePrice':
#         train[i] = boxcox1p(train[i], boxcox_normmax(train[i] + 1))
#         test[i] = boxcox1p(test[i], boxcox_normmax(train[i] + 1))
        train[i] = np.log1p(train[i])
        test[i] =  np.log1p(test[i])

Index(['MiscVal', 'PoolArea', 'LotArea', '3SsnPorch', 'LowQualFinSF',
       'KitchenAbvGr', 'BsmtFinSF2', 'ScreenPorch', 'BsmtHalfBath',
       'EnclosedPorch', 'MasVnrArea', 'OpenPorchSF', 'TotalLivingSF',
       'SalePrice', 'BsmtFinSF1', 'WoodDeckSF', 'TotalBsmtSF', '1stFlrSF',
       'GrLivArea', 'TotalPorchSF', 'BsmtUnfSF', '2ndFlrSF', 'OverallCond',
       'TotRmsAbvGrd', 'HalfBath', 'Fireplaces', 'BsmtFullBath'],
      dtype='object')
MiscVal          24.451640
PoolArea         14.813135
LotArea          12.195142
3SsnPorch        10.293752
LowQualFinSF      9.002080
KitchenAbvGr      4.483784
BsmtFinSF2        4.250888
ScreenPorch       4.117977
BsmtHalfBath      4.099186
EnclosedPorch     3.086696
MasVnrArea        2.674865
OpenPorchSF       2.361912
TotalLivingSF     2.161744
SalePrice         1.880941
BsmtFinSF1        1.683771
WoodDeckSF        1.539792
TotalBsmtSF       1.522688
1stFlrSF          1.375342
GrLivArea         1.365156
TotalPorchSF      1.101180
BsmtUnfSF    

In [8]:
import seaborn as sns
from scipy.stats import norm
from scipy import stats

train['SalePrice'] = np.log(train['SalePrice'])
train_y = train["SalePrice"].copy()


In [9]:
skew(train_y)

0.1212103673013655

In [10]:
numeric_dtypes = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
numerics2 = []
for i in train.columns:
    if train[i].dtype in numeric_dtypes:
        numerics2.append(i)
skew_features = train[numerics2].apply(lambda x: skew(x)).sort_values(ascending=False)

high_skew = skew_features[skew_features > 0.2]
skew_index = high_skew.index
print(skew_index)
print(high_skew)

Index(['PoolArea', '3SsnPorch', 'LowQualFinSF', 'MiscVal', 'BsmtHalfBath',
       'KitchenAbvGr', 'ScreenPorch', 'BsmtFinSF2', 'EnclosedPorch',
       'HalfBath', 'MasVnrArea', 'BsmtFullBath', '2ndFlrSF', 'Has2ndFloor',
       'LotFrontage', 'OverallQual', 'BedroomAbvGr'],
      dtype='object')
PoolArea         14.348342
3SsnPorch         7.727026
LowQualFinSF      7.452650
MiscVal           5.165390
BsmtHalfBath      3.929022
KitchenAbvGr      3.865437
ScreenPorch       3.147171
BsmtFinSF2        2.521100
EnclosedPorch     2.110104
HalfBath          0.565586
MasVnrArea        0.503014
BsmtFullBath      0.418782
2ndFlrSF          0.289346
Has2ndFloor       0.273762
LotFrontage       0.267547
OverallQual       0.216721
BedroomAbvGr      0.211572
dtype: float64


In [11]:
corr_matrix = train.corr()
corr_matrix["SalePrice"].sort_values(ascending=False)

SalePrice        1.000000
OverallQual      0.817184
TotalLivingSF    0.741065
GrLivArea        0.730254
GarageCars       0.680625
GarageArea       0.650888
1stFlrSF         0.608955
FullBath         0.594771
YearBuilt        0.586570
YearRemodAdd     0.565608
TotRmsAbvGrd     0.539998
Fireplaces       0.508925
OpenPorchSF      0.459948
MasVnrArea       0.413318
LotArea          0.399923
TotalPorchSF     0.375169
TotalBsmtSF      0.372838
GarageYrBlt      0.349014
WoodDeckSF       0.343039
HalfBath         0.320925
BsmtFullBath     0.238945
BedroomAbvGr     0.209044
BsmtUnfSF        0.208233
BsmtFinSF1       0.208099
2ndFlrSF         0.180778
LotFrontage      0.179303
Has2ndFloor      0.150549
ScreenPorch      0.105858
PoolArea         0.069949
3SsnPorch        0.058827
OverallCond     -0.001793
BsmtHalfBath    -0.004890
Id              -0.017942
BsmtFinSF2      -0.030361
LowQualFinSF    -0.054513
MiscVal         -0.067054
KitchenAbvGr    -0.144278
EnclosedPorch   -0.207263
Name: SalePr

In [12]:
num_att = list(train.select_dtypes([np.number]).columns)

# att = ['OverallQual', 'GrLivArea']

# to_remove = ['BsmtFinSF2', 'MoSold', '3SsnPorch', 'BsmtHalfBath', 'MiscVal', 'LowQualFinSF', 'YrSold',
#             'PoolArea', 'OverallCond', 'MSSubClass', 'ScreenPorch']

to_remove = ['PoolArea', 'BsmtHalfBath', 'MiscVal', 'LowQualFinSF', 'ScreenPorch', '3SsnPorch',
            'BsmtFinSF2', 'MoSold', 'YrSold',
            'GarageArea', 'TotalBsmtSF']

to_remove = ['PoolArea', 'BsmtHalfBath', 'MiscVal', 'LowQualFinSF', 'ScreenPorch', '3SsnPorch',
            'BsmtFinSF2']

for t in to_remove:
    num_att.remove(t)
num_att.remove('Id')
num_att.remove('SalePrice')

# num_att = ['OverallQual', 'GrLivArea', 'GarageCars', '1stFlrSF', 'FullBath', 'YearBuilt']

print(num_att)

['LotFrontage', 'LotArea', 'OverallQual', 'OverallCond', 'YearBuilt', 'YearRemodAdd', 'MasVnrArea', 'BsmtFinSF1', 'BsmtUnfSF', 'TotalBsmtSF', '1stFlrSF', '2ndFlrSF', 'GrLivArea', 'BsmtFullBath', 'FullBath', 'HalfBath', 'BedroomAbvGr', 'KitchenAbvGr', 'TotRmsAbvGrd', 'Fireplaces', 'GarageYrBlt', 'GarageCars', 'GarageArea', 'WoodDeckSF', 'OpenPorchSF', 'EnclosedPorch', 'TotalLivingSF', 'TotalPorchSF', 'Has2ndFloor']


In [13]:
cat_att = ['BldgType', 'CentralAir', 'Foundation', 
           'PavedDrive', 'SaleCondition']
cat_att = ['Neighborhood', 'BldgType', 'Heating', 'HeatingQC', 'CentralAir', 
           'Street', 'LotShape', 'LandContour', 'LotConfig',
           'LandSlope', 'Condition1', 'HouseStyle', 'RoofStyle', 
           'ExterQual', 'ExterCond', 'Foundation', 'PavedDrive', 'SaleCondition',
           'KitchenQual', 'BsmtQual', 'MoSold', 'YrSold', 'MSSubClass',
           'SaleType', 'Electrical', 'BsmtExposure', 'FireplaceQu',
           'Exterior1st', 'MSZoning', 'Alley',
           'Condition2', 'RoofMatl', 'Exterior2nd', 'MasVnrType', 'BsmtCond',
           'BsmtFinType1', 'BsmtFinType2', 'Functional',
           'GarageType', 'GarageFinish', 'GarageQual', 'GarageCond',
           'Fence', 'MiscFeature']

# TO ADD = Exterior1st, KitchenQual, Functional, SaleType, Alley, Condition2

# cat_att = ['Neighborhood', 'BldgType']
print(cat_att)
print(len(cat_att))
print(len(set(cat_att)))

['Neighborhood', 'BldgType', 'Heating', 'HeatingQC', 'CentralAir', 'Street', 'LotShape', 'LandContour', 'LotConfig', 'LandSlope', 'Condition1', 'HouseStyle', 'RoofStyle', 'ExterQual', 'ExterCond', 'Foundation', 'PavedDrive', 'SaleCondition', 'KitchenQual', 'BsmtQual', 'MoSold', 'YrSold', 'MSSubClass', 'SaleType', 'Electrical', 'BsmtExposure', 'FireplaceQu', 'Exterior1st', 'MSZoning', 'Alley', 'Condition2', 'RoofMatl', 'Exterior2nd', 'MasVnrType', 'BsmtCond', 'BsmtFinType1', 'BsmtFinType2', 'Functional', 'GarageType', 'GarageFinish', 'GarageQual', 'GarageCond', 'Fence', 'MiscFeature']
44
44


In [14]:
train2 = train[num_att + cat_att]
# train2.info()
test2 = test[num_att + cat_att]
# test2.info()

merged_df = pd.concat([train2, test2])
merged_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2919 entries, 0 to 1458
Data columns (total 73 columns):
LotFrontage      2919 non-null float64
LotArea          2919 non-null float64
OverallQual      2919 non-null int64
OverallCond      2919 non-null float64
YearBuilt        2919 non-null int64
YearRemodAdd     2919 non-null int64
MasVnrArea       2919 non-null float64
BsmtFinSF1       2919 non-null float64
BsmtUnfSF        2919 non-null float64
TotalBsmtSF      2919 non-null float64
1stFlrSF         2919 non-null float64
2ndFlrSF         2919 non-null float64
GrLivArea        2919 non-null float64
BsmtFullBath     2919 non-null float64
FullBath         2919 non-null int64
HalfBath         2919 non-null float64
BedroomAbvGr     2919 non-null int64
KitchenAbvGr     2919 non-null float64
TotRmsAbvGrd     2919 non-null float64
Fireplaces       2919 non-null float64
GarageYrBlt      2919 non-null float64
GarageCars       2919 non-null float64
GarageArea       2919 non-null float64
WoodDe

In [15]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder

num_pipeline = Pipeline([
        ('imputer', SimpleImputer(strategy="median")),
        ('std_scaler', StandardScaler()),
    ])
train_prepared = num_pipeline.fit_transform(train[num_att])
test_prepared  = num_pipeline.transform(test[num_att])


full_pipeline = ColumnTransformer([
        ("num", num_pipeline, num_att),
        ("cat", OneHotEncoder(), cat_att),
    ])

train_test_full_prepared = full_pipeline.fit_transform(merged_df)
train_full_prepared = full_pipeline.transform(train2)
test_full_prepared = full_pipeline.transform(test2)


In [16]:
print(train_prepared.shape)
print(test_prepared.shape)
print(train_full_prepared.shape)
print(test_full_prepared.shape)

(1460, 29)
(1459, 29)
(1460, 322)
(1459, 322)


In [17]:
print(np.arange(0.00001, 0.00007, 0.00001))

[1.e-05 2.e-05 3.e-05 4.e-05 5.e-05 6.e-05]


In [21]:
from numpy import arange
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RepeatedKFold
from xgboost import XGBRegressor
cv = RepeatedKFold(n_splits=10, n_repeats=2, random_state=1)
model = XGBRegressor(learning_rate=0.01,n_estimators=3460,
                                     max_depth=3, min_child_weight=0,
                                     gamma=0, subsample=0.7,
                                     colsample_bytree=0.7,
                                     objective='reg:squarederror', nthread=-1,
                                     scale_pos_weight=1, seed=42)

grid = dict()
grid['reg_alpha'] = np.arange(0.00001, 0.00005, 0.00001)
# define search
search = GridSearchCV(model, grid, scoring='neg_mean_squared_error', cv=cv, n_jobs=-1, verbose=3)
# perform the search
results = search.fit(train_full_prepared, train_y)
# summarize
print('MAE: %.3f' % results.best_score_)
print('Config: %s' % results.best_params_)




Fitting 20 folds for each of 4 candidates, totalling 80 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done   8 tasks      | elapsed:   13.2s
[Parallel(n_jobs=-1)]: Done  80 out of  80 | elapsed:  1.4min finished


MAE: -0.014
Config: {'reg_alpha': 1e-05}


In [22]:
cvres = results.cv_results_
for mean_score, params in zip(cvres["mean_test_score"], cvres["params"]):
    print(np.sqrt(-mean_score), params)
    
# 12483177514831173

0.11996694327554766 {'reg_alpha': 1e-05}
0.12008881175586036 {'reg_alpha': 2e-05}
0.11997357497806935 {'reg_alpha': 3.0000000000000004e-05}
0.11999004380345854 {'reg_alpha': 4e-05}


In [ ]:
#for tuning parameters
#parameters_for_testing = {
#    'colsample_bytree':[0.4,0.6,0.8],
#    'gamma':[0,0.03,0.1,0.3],
#    'min_child_weight':[1.5,6,10],
#    'learning_rate':[0.1,0.07],
#    'max_depth':[3,5],
#    'n_estimators':[10000],
#    'reg_alpha':[1e-5, 1e-2,  0.75],
#    'reg_lambda':[1e-5, 1e-2, 0.45],
#    'subsample':[0.6,0.95]  
#}

In [23]:
housing_predictions = results.predict(train_full_prepared)

In [24]:
print(housing_predictions[0])
print(train_y[0])

12.221592
12.247694320220994


In [25]:
from sklearn.metrics import mean_squared_error

train_y_normal = np.e**train_y
# train_y_normal = inv_boxcox1p(train_y, boxcox_normmax(train['SalePrice'] + 1))

pred_normal = np.e**housing_predictions
# pred_normal = inv_boxcox1p(housing_predictions, boxcox_normmax(train['SalePrice'] + 1))

forest_mse = mean_squared_error(train_y_normal, pred_normal)
forest_rmse = np.sqrt(forest_mse)
forest_rmse

9419.51668876332

In [26]:
print(train_y_normal[0])
print(pred_normal[0])

208500.0
203128.0


In [27]:
# y_pred = forest_reg.predict(test_full_prepared)
# y_pred = grid_search.predict(test_full_prepared)
y_pred = results.predict(test_full_prepared)


y_pred_normal = np.e**y_pred
# y_pred_normal = inv_boxcox1p(y_pred, boxcox_normmax(train['SalePrice'] + 1))
sub = pd.DataFrame({'Id': test['Id'], 'SalePrice': y_pred_normal})

In [28]:
sub.to_csv("data/submission_script59_xgb_log1p.csv", index=False)

In [80]:
sub.shape

(1459, 2)

In [20]:
from numpy import arange
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RepeatedKFold
from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error

model = XGBRegressor(learning_rate=0.01,n_estimators=3460,
                                     max_depth=3, min_child_weight=0,
                                     gamma=0, subsample=0.7,
                                     colsample_bytree=0.7,
                                     objective='reg:squarederror', nthread=-1,
                                     scale_pos_weight=1, seed=42, reg_alpha=0.00006)

results = model.fit(train_full_prepared, train_y)

housing_predictions = model.predict(train_full_prepared)
train_y_normal = np.e**train_y
pred_normal = np.e**housing_predictions
xgb_mse = mean_squared_error(train_y_normal, pred_normal)
xgb_rmse = np.sqrt(xgb_mse)
xgb_rmse

9403.733820189362

In [21]:
y_pred = results.predict(test_full_prepared)


y_pred_normal = np.e**y_pred
# y_pred_normal = inv_boxcox1p(y_pred, boxcox_normmax(train['SalePrice'] + 1))
sub = pd.DataFrame({'Id': test['Id'], 'SalePrice': y_pred_normal})

In [22]:
sub.to_csv("data/submission_script59_xgb_log1p_3.csv", index=False)